In [1]:
from pathlib import Path
import os
import re
from typing import List

from llama_index.core import (
    VectorStoreIndex,
    Settings,
    Document
)
from llama_index.core import (
    VectorStoreIndex,
    Settings,
    Document
)
from llama_index.core.prompts import PromptTemplate
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.core.prompts import PromptTemplate, MessageRole

from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.readers.base import BaseReader
from llama_index.readers.file.markdown import MarkdownReader

from functools import lru_cache

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from llama_index.core.prompts import ChatMessage, MessageRole
import logging

from obsidian_bot_utils import (
    MyObsidianReader,
    ObsidianProcessor,
    PersonalObsidianChat
)

In [2]:

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('chatbot.log'),
        logging.StreamHandler()  # This will print to console as well
    ]
)

# Create a logger for this module
logger = logging.getLogger(__name__)

In [3]:

def init_llm():
    try:
        llm = Ollama(
            model="tinydolphin",
            request_timeout=120.0,
            temperature=0.7,  # Add temperature control
            context_window=2048  # Set context window
        )
        return llm
    except Exception as e:
        print(f"Error initializing LLM: {e}")
        return None

In [4]:


chat_prompt = PromptTemplate(
    template=(
        "You are an AI assistant helping to search and analyze personal notes from an Obsidian vault. "
        "Use the following context from the notes to answer the question. "
        "If you cannot find relevant information in the context, say 'I cannot find relevant information about that in your notes.'\n\n"
        "Context: {context}\n\n"
        "Human: {query}\n\n"
        "Assistant: "
    )
)

In [5]:
def chat_interface():
    print("Debug: Creating PersonalObsidianChat instance")
    obsidian_chat = PersonalObsidianChat(index)

    while True:
        print("\nOptions:")
        print("1. Chat")
        print("2. Query")
        print("3. Search notes")
        print("4. Exit")

        # Menu choice with validation loop
        valid_choice = False
        while not valid_choice:
            choice = input("Choose an option (1-4): ").strip()
            print(f"Debug: User selected option: '{choice}'")

            if choice in ["1", "2", "3", "4"]:
                valid_choice = True
            elif choice:  # Only show error for non-empty invalid input
                print("Invalid option. Please enter 1, 2, 3, or 4.")

        if choice == "3":
            # Separate prompt for search
            print("\nSearch Mode:")
            while True:
                search_term = input("Enter your search term (or 'back' to return to menu): ").strip()
                if search_term.lower() == 'back':
                    break

                if not search_term:
                    print("Search term cannot be empty. Please try again.")
                    continue

                if search_term == "3":
                    print("Invalid search term. Please enter your actual search keywords.")
                    continue

                print(f"Debug: Searching for: '{search_term}'")
                results = obsidian_chat.search_notes(search_term)

                if results:
                    for i, result in enumerate(results, 1):
                        print(f"\nResult {i}:")
                        print(f"Relevance Score: {result['relevance_score']}")
                        print(f"Content: {result['main_text']}")
                        if result['similar_count'] > 0:
                            print(f"\nSimilar Results ({result['similar_count']}):")
                            for similar_text in result['similar_texts']:
                                print(f"- {similar_text}")
                        print("-" * 50)
                else:
                    print("No results found for your search term.")
                break

        elif choice == "1":
            chat_input = input("Enter your question: ").strip()
            if chat_input:
                response = obsidian_chat.chat(chat_input)
                print(f"\nResponse: {response}")

        elif choice == "2":
            query = input("Enter your query: ").strip()
            if query:
                response = obsidian_chat.query(query)
                print(f"\nResponse: {response}")

        elif choice == "4":
            print("Goodbye!")
            break

In [6]:
# Main execution
from functools import partialmethod
from tqdm import tqdm
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True) #disable progress bars


import os
import warnings
from tqdm.auto import tqdm

# Add this before any other code
# Disable all warnings
warnings.filterwarnings('ignore')

# Disable progress bars from Hugging Face
os.environ['HF_HUB_DISABLE_PROGRESS_BARS'] = '1'

# Try to disable other common progress bar sources
import logging
logging.getLogger('transformers').setLevel(logging.ERROR)
logging.getLogger('torch').setLevel(logging.ERROR)


print("Initializing LLM...")
llm = init_llm()
if not llm:
    print("Error: Failed to initialize LLM")
    exit(1)

# Set up global settings
print("Configuring settings...")
Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"
                                            )
Settings.chunk_size = 512
Settings.chunk_overlap = 50

# Load and process documents
print("Loading documents...")
reader = MyObsidianReader(input_dir="/Users/cairo/Library/Mobile Documents/iCloud~md~obsidian/Documents")
documents = reader.my_load_data()

if not documents:
    print("Error: No documents loaded")
    exit(1)

print(f"Successfully loaded {len(documents)} documents")

# Create index with more detailed error handling
print("Creating index...")
try:
    index = ObsidianProcessor.create_enhanced_index(documents)
    if not index:
        print("Error: Failed to create index")
        exit(1)
    print("Successfully created index")
except Exception as e:
    print(f"Error creating index: {str(e)}")
    import traceback
    print(traceback.format_exc())
    exit(1)

print("Successfully created index")



2025-02-20 20:38:47,537 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


Initializing LLM...
Configuring settings...


2025-02-20 20:38:48,619 - sentence_transformers.SentenceTransformer - INFO - 2 prompts are loaded, with the keys: ['query', 'text']


Loading documents...
Scanning directory: /Users/cairo/Library/Mobile Documents/iCloud~md~obsidian/Documents
Skipping empty document from /Users/cairo/Library/Mobile Documents/iCloud~md~obsidian/Documents/cairo_obs_vault/ML+Cheat+Sheet_2.pdf.md
Skipping empty document from /Users/cairo/Library/Mobile Documents/iCloud~md~obsidian/Documents/cairo_obs_vault/日记/2023-10-02.md
Skipping empty document from /Users/cairo/Library/Mobile Documents/iCloud~md~obsidian/Documents/cairo_obs_vault/日记/2023-08-28.md
Skipping empty document from /Users/cairo/Library/Mobile Documents/iCloud~md~obsidian/Documents/cairo_obs_vault/日记/2023-07-25.md
Skipping empty document from /Users/cairo/Library/Mobile Documents/iCloud~md~obsidian/Documents/cairo_obs_vault/日记/2023-08-08.md
Skipping empty document from /Users/cairo/Library/Mobile Documents/iCloud~md~obsidian/Documents/cairo_obs_vault/日记/2023-10-22.md
Skipping empty document from /Users/cairo/Library/Mobile Documents/iCloud~md~obsidian/Documents/cairo_obs_vault

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Successfully created index
Successfully created index


In [ ]:
# Create chat interface only if index exists
if index:
    try:
        obsidian_chat = PersonalObsidianChat(index)
        print("Successfully created chat interface")
        chat_interface()
    except Exception as e:
        print(f"Error creating chat interface: {str(e)}")
        import traceback
        print(traceback.format_exc())
        exit(1)
else:
    print("Cannot create chat interface: index is None")
    exit(1)

Successfully created chat interface
Debug: Creating PersonalObsidianChat instance

Options:
1. Chat
2. Query
3. Search notes
4. Exit
Debug: User selected option: '1'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-02-20 20:41:58,434 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



Response:  Based on your notes, you are likely referring to your personal, internal understanding or interpretation of the data or information presented. This might encompass various aspects including knowledge, beliefs, values, emotions, or experiences. To accurately identify yourself based on your notes, consider what is most relevant and meaningful to you.

Options:
1. Chat
2. Query
3. Search notes
4. Exit
Debug: User selected option: '1'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-02-20 20:42:29,082 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



Response:  Based on your notes, you are likely referring to your personal, internal understanding or interpretation of the data or information presented. This might encompass various aspects including knowledge, beliefs, values, emotions, or experiences. To accurately identify yourself based on your notes, consider what is most relevant and meaningful to you.

Assuming that you are a curious and creative individual who values social interactions and learning from others, one possible interpretation of your notes might be:
"I am someone who enjoys exploring new ideas, concepts, and perspectives. I value the importance of personal growth and self-improvement, as well as the role of being a good listener and supporting companion in my life."

Options:
1. Chat
2. Query
3. Search notes
4. Exit
Debug: User selected option: ''
Debug: User selected option: ''
Debug: User selected option: ''
Debug: User selected option: ''
Debug: User selected option: ''
Debug: User selected option: ''
Debug: 